# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

In [3]:
gsd_file = "/Users/jimmy/cmelab/data/ITIC/workspace/d753f6a7bda4d15489a5f38c1e24a17e/trajectory.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    

ref_distance = 3.563594872561358

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

box = start_snap.configuration.box[:3]
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1)
comp.visualize().show()

#box = end_snap.configuration.box[:3]
#unwrapped_positions = end_snap.particles.position + end_snap.particles.image * box
#unwrap_snap = deepcopy(end_snap)
#unwrap_snap.particles.position = unwrapped_positions
#unwrap_snap.particles.types = [amber_dict[i].symbol for i in end_snap.particles.types]
#comp = from_snapshot(unwrap_snap, scale=0.1)
#comp.visualize().show()

/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
print(start_snap.configuration.box)
print(end_snap.configuration.box)

[389.9132 389.9132 389.9132   0.       0.       0.    ]
[78.29458 78.29458 78.29458  0.       0.       0.     ]


/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


the indexes below came from viewing the single molecule trajectory in vmd and clicking on the molecules that I like for my chromos. this cell visualizes those chosen

In [6]:
chromo_ids = np.array([0,1,2,4,6,7,10,11,12,13,19,20,22,23,24,25,27,28,29,30,31,32,91,92,93,94,97,98,99,100,101,102,161,162,163,165,166,168,169,170,171,172,178,179,180,181,177,175,176,174,173,17,15,18,16,14,13])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Trying to get homo and lumo for a single molecule of itic below. using functions in execute_qcc.py. here we use all molecule ids from start_snap.particles.N to get homo lumos for sinlge molecule gsd

displayed : 

HOMO-1, HOMO, LUMO, LUMO+1 above. 

In [ ]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(start_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


In [ ]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(end_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)

In [ ]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


below i want to get the homolumos for just that back bone as taken from a single molecule gsd

In [ ]:
qcc_input = eqcc.write_qcc_inp(end_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


above this is just the single molecule homo lumo calcs. below i want to bring the build created gsd and get pairwise calcs. the mbuild stuff is being weird . going to just use a two molecule gsd from plankton to begin

In [ ]:
chromo2_ids = chromo_ids +186

In [ ]:
aaids=[]

In [ ]:
aaids.append(chromo_ids)
aaids.append(chromo2_ids)

In [ ]:
print(aaids)

In [ ]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, start_snap, aaid, "acceptor", amber_dict))

In [ ]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=100)
print(f"There are {len(qcc_pairs)} chromophore pairs")

In [ ]:
print(qcc_pairs)

In [ ]:
i =  0# try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

In [ ]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")

In [ ]:
%%time
data = eqcc.singles_homolumo(chromo_list, "singles.txt")

In [ ]:
%%time
eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")

In [ ]:
print(chromo_list)

In [ ]:
i = 0
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")